# Modeling a Pin-Cell with OpenMC
This notebook is an example for NSE 457/557 reactor lab. In it, I am going to model a pin cell with reflective boundary conditions. This can be a useful reference for the OpenMC portion of lab 1. 

Another tip: I don't have any of this memorized. I like to check the [Python API reference documentation](https://docs.openmc.org/en/stable/pythonapi/index.html) whenever I'm doing something new.

This notebook is inspired by (and is based on) the example notebook of [modeling a pin cell](https://nbviewer.org/github/openmc-dev/openmc-notebooks/blob/main/pincell.ipynb) from the OpenMC documentation. 

First, I'll import relevant packages and use the inline matplotlib magic. 

In [ ]:
%matplotlib inline
import openmc

### Basics of an OpenMC Input 

In constructing an OpenMC input, we care about some conditions:
* The problem geometry, which includes:
  * surfaces (planes, spheres, cylinders) that make up problem
  * cells that are combinations of surfaces, and potentially
  * universes, which is a collection of cells that can be repeated or collectively define a problem.
* The problem materials
* The problem settings. This might include:
  * the type of problem (fixed source or eigenvalue)
  * how many particles to simulate (potentially also generations)
  * the temperature of the problem
* Results, in Monte Carlo we get these through tallies

#### Requirements for an OpenMC run
* materials.xml
* geometry.xml
* settings.xml
* _tallies.xml_
* _plots.xml_

For each of these we are going to go through an use OpenMC's Python API to define materials, geometry, and settings to set up a pin cell problem. 

## Defining Materials

Materials in OpenMC are defined as a set of nuclides with specified atom/weight fractions. To begin, we will create a material by making an instance of the `Material` class. 

In OpenMC, many objects, including materials, are identified by a "unique ID" that is simply just a positive integer. These IDs are used when exporting XML files that the solver reads in. They also appear in the output and can be used for identification. Since an integer ID is not very useful by itself, we can also give a material a `name` as well. Let's create a material definition for UO$_2$ that we can use later. 

In [ ]:
uo2 = openmc.Material(1, "uo2")
print(uo2)

Right now we have a UO$_2$ material, but nothing is in the definition that indicates its composition. Let's add some nuclides to our `uo2` material. The `Material` object has a method `add_nuclide()` whose first argument is the name of the nuclide and second argument is the atom or weight fraction.

In [ ]:
help(uo2.add_nuclide)

We see that by default it assumes we want an atom fraction.

In [ ]:
# Add nuclides to uo2
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)

Now we need to assign a total density to the material. We'll use the `set_density` for this.

In [ ]:
uo2.set_density('g/cm3', 10.97)

Another thing we can do is specify an element, rather than an isotope. This will automatically select the natural abundance of that elemeent. 

In [ ]:
uranium = openmc.Material(name="uranium")
uranium.add_element('U', 1.0)
uranium.set_density('g/cm3', 19.1)
print(uranium)

See how the natural abundances were included for elemental U? This is a nice shortcut! 

Note: Sometimes we get a material specification where all the nuclide densities are in units of atom/b-cm. In this case, we'll want the density to be the sum of the constituents. In that case, run `mat.set_density('sum')`.

Let's now create materials for the clad, air, and graphite. Note the use of `add_element()` for these materials where I specify elemental compositions rather than isotopics.

In [ ]:
T6 = openmc.Material(name="T6")
T6.add_element('Al', 98, percent_type='wo')
T6.add_element('Mg', 1.0, percent_type='wo')
T6.add_element('Si', 0.4, percent_type='wo')
T6.add_element('Fe', 0.2, percent_type='wo')
T6.add_element('Cr', 0.05, percent_type='wo')
T6.add_element('Cu', 0.2, percent_type='wo')
T6.add_element('Al', 0.05, percent_type='wo')
T6.add_element('Zn', 0.05, percent_type='wo')
T6.add_element('Ti', 0.025, percent_type='wo')
T6.add_element('Mn', 0.025, percent_type='wo')
T6.set_density('g/cm3', 2.7)

air = openmc.Material(name="air")
air.add_element('C', 0.0124, percent_type='wo')
air.add_element('N', 75.5267, percent_type='wo')
air.add_element('O', 23.1781, percent_type='wo')
air.add_element('Ar', 1.2827, percent_type='wo')
air.set_density('g/cm3', .00120479)

graphite = openmc.Material(name="graphite")
graphite.add_element('C', 1.0)
graphite.set_density('g/cm3', 1.7)

In [ ]:
print(T6)

When we go to run the transport solver in OpenMC, it is going to look for a `materials.xml` file. Thus far, we have only created objects in memory. To actually create a `materials.xml` file, we need to instantiate a `Materials` collection and export it to XML.

In [ ]:
materials = openmc.Materials([T6, air, graphite])

Note that `Materials` is actually a subclass of Python's built-in `list`, so we can use methods like `append()`, `insert()`, `pop()`, etc.

In [ ]:
materials = openmc.Materials()
materials.append(uo2)
materials += [T6, air, graphite]
isinstance(materials, list)
print(materials)

Finally, we can create the XML file with the `export_to_xml()` method. In a Jupyter notebook, we can run a shell command by putting `!` before it, so in this case we are going to display the `materials.xml` file that we created.

In [ ]:
materials.export_to_xml()
!cat materials.xml

In our jupyterlab tab we should see a new file generated for the materials configuration. 

### Element Expansion

Did you notice something really cool that happened to our air elements? OpenMC automatically turned it into a list of nuclides when it exported it! The way this feature works is as follows:

- First, it checks whether `Materials.cross_sections` has been set, indicating the path to a `cross_sections.xml` file.
- If `Materials.cross_sections` isn't set, it looks for the `OPENMC_CROSS_SECTIONS` environment variable.
- If either of these are found, it scans the file to see what nuclides are actually available and will expand elements accordingly.

In your lab using the campus cluster, the `OPENMC_CROSS_SECTIONS` environment variable is set in your configuration already. I strongly recommend using this and not changing cross sections (if you do choose to change them, then please state it explicitly in your report). 

### The `cross_sections.xml` file

Let's use the `OPENMC_CROSS_SECTIONS` environment variable to inspect our file. 

The `cross_sections.xml` tells OpenMC where it can find nuclide cross sections and $S(\alpha,\beta)$ tables. It serves the same purpose as MCNP's `xsdir` file and Serpent's `xsdata` file if you are familiar with these tools. As mentioned, this can be set either by the `OPENMC_CROSS_SECTIONS` environment variable or the `Materials.cross_sections` attribute.

Let's have a look at what's inside this file:

In [ ]:
!cat $OPENMC_CROSS_SECTIONS | head -n 10
print('    ...')
!cat $OPENMC_CROSS_SECTIONS | tail -n 10

We can see that these are .h5 files, which are a standard format for OpenMC. Note that if we wanted to download cross sections from nndc we would need to modify them to this format. You can find out a bit more about that [here](https://docs.openmc.org/en/v0.12.1/usersguide/cross_sections.html#manually-creating-a-library-from-ace-files)

## Defining Geometry

At this point, we have four materials defined (air, graphite, T6, and uo2), exported to XML, and ready to be used in our model. 

To finish our model, we need to define the geometric arrangement of materials. 

OpenMC represents physical volumes using constructive solid geometry (CSG), also known as combinatorial geometry. The object that allows us to assign a material to a region of space is called a `Cell` (same concept in MCNP, for those familiar). In order to define a region that we can assign to a cell, we must first define surfaces which bound the region. A *surface* is a locus of zeros of a function of Cartesian coordinates $x$, $y$, and $z$, e.g.

- A plane perpendicular to the x axis: $x - x_0 = 0$
- A cylinder parallel to the z axis: $(x - x_0)^2 + (y - y_0)^2 - R^2 = 0$
- A sphere: $(x - x_0)^2 + (y - y_0)^2 + (z - z_0)^2 - R^2 = 0$

Between those three classes of surfaces (planes, cylinders, spheres), one can construct a wide variety of models. It is also possible to define cones and general second-order surfaces (tori are not currently supported).

Note that defining a surface is not sufficient to specify a volume -- in order to define an actual volume, one must reference the half-space of a surface. A surface *half-space* is the region whose points satisfy a positive or negative inequality of the surface equation. For example, for a sphere of radius one centered at the origin, the surface equation is $f(x,y,z) = x^2 + y^2 + z^2 - 1 = 0$. Thus, we say that the negative half-space of the sphere, is defined as the collection of points satisfying $f(x,y,z) < 0$, which one can reason is the inside of the sphere. Conversely, the positive half-space of the sphere would correspond to all points outside of the sphere.

Let's go ahead and create a sphere and confirm that what I'm saying is true.

In [ ]:
sphere = openmc.Sphere(r=1.0)

Note that by default the sphere is centered at the origin so we didn't have to supply `x0`, `y0`, or `z0` arguments. Strictly speaking, we could have omitted `r` as well since it defaults to one. To get the negative or positive half-space, we simply need to apply the `-` or `+` unary operators, respectively.

(NOTE: Those unary operators are defined by special methods: `__pos__` and `__neg__` in this case).

In [ ]:
inside_sphere = -sphere
outside_sphere = +sphere

Now let's see if `inside_sphere` actually contains points inside the sphere:

In [ ]:
print((0,0,0) in inside_sphere, (0,0,2) in inside_sphere)
print((0,0,0) in outside_sphere, (0,0,2) in outside_sphere)

Everything works as expected! Now that we understand how to create half-spaces, we can create more complex volumes by combining half-spaces using Boolean operators: `&` (intersection), `|` (union), and `~` (complement). For example, let's say we want to define a region that is the top part of the sphere (all points inside the sphere that have $z > 0$.

In [ ]:
z_plane = openmc.ZPlane(0)
northern_hemisphere = -sphere & +z_plane

For many regions, OpenMC can automatically determine a bounding box. To get the bounding box, we use the `bounding_box` property of a region, which returns a tuple of the lower-left and upper-right Cartesian coordinates for the bounding box:

In [ ]:
northern_hemisphere.bounding_box

Now that we see how to create volumes, we can use them to create a cell.

In [ ]:
cell = openmc.Cell()
cell.region = northern_hemisphere

# or...
cell = openmc.Cell(region=northern_hemisphere)

By default, the cell is not filled by any material (void). In order to assign a material, we set the `fill` property of a `Cell`.

In [ ]:
cell.fill = graphite

### Universes and in-line plotting

Universes are collections of cells that we use to define our problem. Universes can be used to represent portions of the problem or the entire problem definition. They can also be repeatable units that we use in say, a lattice definition. 

Another benefit of creating a universe is that we can visualize our geometry while we're creating it.

In [ ]:
universe = openmc.Universe()
universe.add_cell(cell)

# this also works
universe = openmc.Universe(cells=[cell])

The `Universe` object has a `plot` method that will display our the universe as current constructed:

In [ ]:
universe.plot(width=(2.0, 2.0), origin=(0.0, 0.0, 0.1))

By default, the plot will appear in the $x$-$y$ plane. We can change that with the `basis` argument.

In [ ]:
universe.plot(width=(2.0, 2.0), basis='xz')

If we have particular fondness for, say, fuchsia, we can tell the `plot()` method to make our cell that color.

In [ ]:
universe.plot(width=(2.0, 2.0), basis='xz',
              colors={cell: 'fuchsia'})

### Pin cell geometry

We now have enough knowledge to create our pin-cell. We need four surfaces to define the fuel and clad:

1. The radius of the air gap -- a cylinder parallel to the z axis
2. The outer surface of the inner clad region -- same as above
3. The outer surface of the fuel -- same as above
4. The outer surface of the outer clad region -- same as above

These four surfaces will all be instances of `openmc.ZCylinder`, each with a different radius according to the specification.

In [ ]:
air_gap_radius = openmc.ZCylinder(r=0.46)
inner_clad_outer_surface = openmc.ZCylinder(r=0.61875)
fuel_outer_surface = openmc.ZCylinder(r=1.76125)
outer_clad_outer_radius = openmc.ZCylinder(r=1.92)

With the surfaces created, we can now take advantage of the built-in operators on surfaces to create regions for the fuel, the gap, and the clad:

In [ ]:
air_region = -air_gap_radius
inner_clad_region = +air_gap_radius & -inner_clad_outer_surface
fuel_region = +inner_clad_outer_surface & -fuel_outer_surface
outer_clad_region = +fuel_outer_surface & -outer_clad_outer_radius

Now we can create corresponding cells that assign materials to these regions. As with materials, cells have unique IDs that are assigned either manually or automatically. Note that the gap cell doesn't have any material assigned (it is void by default).

In [ ]:
# Here I define the center as gap because I've already used air as the name for my material
gap = openmc.Cell(name='air gap')
gap.fill = air
gap.region = air_region

inner_clad = openmc.Cell(name='inner clad')
inner_clad.fill = T6
inner_clad.region = inner_clad_region

fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region

outer_clad = openmc.Cell(name='outer clad')
outer_clad.fill = T6
outer_clad.region = outer_clad_region

Finally, we need to handle the graphite outside of our fuel pin. To do this, we create x- and y-planes that bound the geometry.

In [ ]:
pitch = 15.0
length = 135.0
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')

Note here that I only define x and y planes. For your. lab you'll need to use zplanes to include the air gap behind the pin. 

The graphite region is going to be everything outside of the clad outer radius and within the box formed as the intersection of four half-spaces.

In [ ]:
graphite_region = +left & -right & +bottom & -top & +outer_clad_outer_radius

moderator = openmc.Cell(name='moderator')
moderator.fill = graphite
moderator.region = graphite_region

OpenMC also includes a factory function that generates a rectangular prism that could have made our lives easier. 

Note that there's a factory function for a bounded prism called a `RectangularParallelpiped()` that is mentioned in the lab prompt as well. 

In [ ]:
box = openmc.model.RectangularPrism(width=pitch, height=pitch,
                                    boundary_type='reflective')
type(box)

Pay attention here -- the object that was returned is NOT a surface. It is actually the intersection of four surface half-spaces, just like we created manually before. Thus, we don't need to apply the unary operator (`-box`). Instead, we can directly combine it with `+clad_or`.

However, we still can still use `-box` if we'd like. 

In [ ]:
water_region = -box & +outer_clad_outer_radius

The final step is to assign the cells we created to a universe and tell OpenMC that this universe is the "root" universe in our geometry. The `Geometry` is the final object that is actually exported to XML.

In [ ]:
root_universe = openmc.Universe(cells=(gap, inner_clad, fuel, outer_clad, moderator))

geometry = openmc.Geometry()
geometry.root_universe = root_universe

# we can also use 
geometry = openmc.Geometry(root_universe)


geometry.export_to_xml()
!cat geometry.xml

Now I also have a `geometry.xml` file that we can see in the left hand side of the pane. 

## Starting source and settings

The Python API has a module ``openmc.stats`` with various univariate and multivariate probability distributions. We can use these distributions to create a starting source using the ``openmc.IndependentSource`` object.

In [ ]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.IndependentSource(space=point)

Note that the OpenMC also has a `space` attribute that can be used to define the spatial distribution of a source. [reference documentation](https://docs.openmc.org/en/v0.12.2/pythonapi/generated/openmc.Source.html)

Now let's create a `Settings` object and give it the source we created along with specifying how many batches and particles we want to run. It is good practice to use low particle counts when you first start to make sure things are working. 

In [ ]:
settings = openmc.Settings()
settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000

In [ ]:
settings.export_to_xml()
!cat settings.xml

We now have all of the required files for an OpenMC simulation. That's enough, but I'd like to do a quick aside about tallying since you'll want to do this in the lab too. 

## User-defined tallies

In this section I will show how one would tally the total, fission, absorption, and (n,$\gamma$) reaction rates for $^{235}$U in the cell containing fuel. Recall that filters allow us to specify *where* in phase-space we want events to be tallied and scores tell us *what* we want to tally:

$$X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int dE}_{\text{filters}} \; \underbrace{f(\mathbf{r},\mathbf{\Omega},E)}_{\text{scores}} \psi (\mathbf{r},\mathbf{\Omega},E)$$

In this case, the *where* is "the fuel cell". So, we will create a cell filter specifying the fuel cell.

In [ ]:
cell_filter = openmc.CellFilter(fuel)

tally = openmc.Tally(1)
tally.filters = [cell_filter]

The *what* is the total, fission, absorption, and (n,$\gamma$) reaction rates in $^{235}$U. By default, if we only specify what reactions, it will gives us tallies over all nuclides. We can use the `nuclides` attribute to name specific nuclides we're interested in.

In [ ]:
tally.nuclides = ['U235']
tally.scores = ['total', 'fission', 'absorption', '(n,gamma)']

Similar to the other files, we need to create a `Tallies` collection and export it to XML.

In [ ]:
tallies = openmc.Tallies([tally])
tallies.export_to_xml()
!cat tallies.xml

## Running OpenMC

Running OpenMC from Python can be done using the `openmc.run()` function. This function allows you to set the number of MPI processes and OpenMP threads, if need be.

In [ ]:
openmc.run()

Great! OpenMC already told us our k-effective. It also spit out a file called `tallies.out` that shows our tallies. This is a very basic method to look at tally data; for more sophisticated methods, see other example notebooks.

In [ ]:
!cat tallies.out

## Geometry plotting

We saw before that we could call the `Universe.plot()` method to show a universe while we were creating our geometry. There is also a built-in plotter in the codebase that is much faster than the Python plotter and has more options. The interface looks somewhat similar to the `Universe.plot()` method. Instead though, we create `Plot` instances, assign them to a `Plots` collection, export it to XML, and then run OpenMC in geometry plotting mode. As an example, let's specify that we want the plot to be colored by material (rather than by cell) and we assign yellow to fuel and blue to water.

In [ ]:
plot = openmc.Plot()
plot.filename = 'pinplot'
plot.width = (pitch, pitch)
plot.pixels = (200, 200)
plot.color_by = 'material'
plot.colors = {uo2: 'yellow', graphite: 'gray'}

With our plot created, we need to add it to a `Plots` collection which can be exported to XML.

In [ ]:
plots = openmc.Plots([plot])
plots.export_to_xml()
!cat plots.xml

Now we can run OpenMC in plotting mode by calling the `plot_geometry()` function. Under the hood this is calling `openmc --plot`.

In [ ]:
openmc.plot_geometry()

Now, we can use functionality from IPython to display the `.png` image inline in our notebook:

In [ ]:
from IPython.display import Image
Image("pinplot.png")

OpenMC also provides us with a method on the `Plot` class that simplifies the workflow.

In [ ]:
plot.to_ipython_image()